In [ ]:
# 在本机未更新至2.0.8的情况下，使用vnpy2.0.8的回测逻辑
import sys
import os
from pathlib import Path
new_version_path = Path(os.getenv('VNPY2.0.8'))
sys.path.insert(0, str(new_version_path))

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
plt.style.use('ggplot')
pd.set_option('display.float_format', '{:.2f}'.format)

from datetime import datetime, timedelta
from typing import Optional
from utility import load_data, strip_digt, comodity_to_vt_symbol, get_output_path


def get_hist_vol(
    commodity: str,
    start: Optional[datetime] = None,
    end: Optional[datetime] = None,
    n: int = 20,
    fast: int = 20,
    slow: int = 60
):
    vt_symbol = comodity_to_vt_symbol(commodity, data_mode='index')
    if not end:
        end = datetime.now()
    if not start:
        start = end - timedelta(days=365 * 10)
        
    df = load_data(vt_symbol, 'd', start, end)
    df["returns"] = np.log(df["close"] / df["close"].shift(1))
    df['hist_vol'] = df['returns'].rolling(n).std() * np.sqrt(252) * 100
    
    df['vol_fast'] = df['hist_vol'].rolling(fast).mean()
    df['vol_slow'] = df['hist_vol'].rolling(slow).mean()
    df = df.dropna()
    return df

def save_hist_vol_img(commodity, **kwargs):
    if 'start' in kwargs:
        start = str(kwargs['start'].year)
    else:
        start = 'default'
        
    if 'end' in kwargs:
        end = str(kwargs['end'].year)
    else:
        end = 'default'
        
    n = kwargs['n'] if 'n' in kwargs else 20
    fast = kwargs['fast'] if 'fast' in kwargs else 20
    slow = kwargs['slow'] if 'slow' in kwargs else 60
    
    folder = f"{start}-{end}-{str(n)}-{str(fast)}-{str(slow)}"
    
    df = get_hist_vol(commodity, **kwargs)
    fig = plt.figure(figsize=(16, 10))
    
    close_plot = plt.subplot(2, 1, 1)
    close_plot.set_title(f"{commodity} close")
    close_plot.plot(df["close"].index, df["close"].values)

    vol_plot = plt.subplot(2, 1, 2)
    vol_plot.set_title(f"{commodity} history volotility")
    # vol_plot.plot(df["hist_vol"].index, df['hist_vol'].values)
    vol_plot.plot(df["vol_fast"].index, df['vol_fast'].values)
    vol_plot.plot(df["vol_slow"].index, df['vol_slow'].values)

    plt.tight_layout() 
    fig.savefig(get_output_path(f"{commodity}.png", 'history_volotility', folder), dpi=150)

In [ ]:
save_hist_vol_img('rb')

In [ ]:
commodity_list = [
    "cu", "al", "zn", "pb", "ni", "sn", "au", "ag", "rb", "hc", "bu", "ru", "sp",
    "m", "y", "a", "b", "p", "c", "cs", "jd", "l", "v", "pp", "j", "jm", "i",
    "SR", "CF", "ZC", "FG", "TA", "MA", "OI", "RM", "SF", "SM"
]
# commodity_list = [
#     "cu", "al"
# ]

for commodity in commodity_list:
    save_hist_vol_img(commodity)